# DNA Sequence Data Explorer

Let's open an .ab1 file, aka trace file, aka chromatogram, using [BioPython](https://biopython.org/https://biopython.org/).

In [148]:
import os
from ipywidgets import widgets

sequence_dropdown = widgets.Dropdown(options=os.listdir('./trace_files'))
display(sequence_dropdown)

Dropdown(options=('black.grainy.mold-ITS1-F.ab1', 'Cordycep-ITS1-F.ab1', 'dense.green.mold-ITS1-F.ab1', 'dense…

Let's take a look at some of the annotation data in the file.

In [141]:
import pandas as pd
from Bio import SeqIO

selected_file = sequence_dropdown.value
selected_file_path = './trace_files/{}'.format(selected_file)

In [149]:
chromatogram = SeqIO.read(selected_file_path, 'abi')

annotations_df = pd.DataFrame.from_dict(chromatogram.annotations,
                                        orient='index',
                                        columns=['Value'])
display(annotations_df)

,Value
sample_well,b'F11'
dye,b'Z-BigDyeV3'
polymer,b'POP7 '
machine_model,b'3730'
run_start,2020-11-25 20:47:19
run_finish,2020-11-25 22:03:24
abif_raw,"{'AEPt1': 11462, 'AEPt2': 11462, 'APFN2': b'KB..."
molecule_type,DNA


The chromatogram data lives in the `abif_raw` row. Let's take a closer look by mapping the keys in `abiif_raw` with the [ABIF File Format spec](docs/ABIF_File_Format.pdf).

In [154]:
abif_raw = annotations_df.loc['abif_raw']['Value']
pd.DataFrame.from_dict({
  'User':                      abif_raw['User1'],
  'Stop Point':                abif_raw['AEPt1'],
  'Start Point':               abif_raw['ASPt1'],
  'Peak Area Ratio':           abif_raw['phAR1'],
  'Peak Spacing':              abif_raw['SPAC3'],
  'Max Quality Value':         abif_raw['phQL1'],
  'Last Successful Analysis':  abif_raw['BCTS1'],
  'Signal Level for Each Dye': abif_raw['S/N%1'],
  'Sample Comment':            abif_raw['CMNT1'],
  'Container ID':              abif_raw['CTID1'],
  'Container Name':            abif_raw['CTNM1'],
  'Container Owner':           abif_raw['CTOw1']},
    orient='index',
    columns=['Raw Data Fields'])

,Raw Data Fields
User,b'genewiz'
Stop Point,11462
Start Point,1695
Peak Area Ratio,-1
Peak Spacing,14.3178
Max Quality Value,99
Last Successful Analysis,b'2020-11-25 22:07:54 -05:00'
Signal Level for Each Dye,"(57, 64, 72, 71)"
Sample Comment,b'2377975-F11-JoshMcGinnis-30-449493217-JM9'
Container ID,b'09A000036479'


### Now let's take a look at the sequence data

In [174]:
sequence = abif_raw['PBAS1'].decode('UTF-8')
sequence_list = list(sequence)

print("Total Bases: {}\n\n{}".format(len(sequence), ' '.join(sequence_list)))

Total Bases: 603

N N N N N N N N N N N N N N N N N N N N C N G N N N T N T N N N A N N A N N N C C C A C N N N N N A N N N N N A N N A G N N G G N N C A N A N A A A A C G C G G G G A N A A G A C N A N T T N N N N N A C C G C C T C C G A A C T C A T T T C T C C T C T N G A T N C N T T T A C N C T T T T C T A G A A T G N T N A T N G G C T G C N N C A T T T T T N C G T N C A C C T A T C G T G N T A A A A C A T T C A A C A T C C T A T C T C T G G N A C T G N T C T C T T A A A C T A A A T T G N N N A N G N N T G T T A T G A N N N N A T C T T A A A A C T T C C N T T N C G C A C C C A G G C A T T C N C C C C C C C T T G C C T C C T C C A N C G T T A T T T C A C G C C T C C C T G C T C N A N T G N C A T T G C A A C C N T C G A C T T C C T G T T G N N N A A N T N N N N A C C N G N G A C N N C T T C T G T C C C C T A N G C C C N G A G G G N N T A T T C G N T A A A G G N G G T T C G A N A G G C T A C G C C N T N C A A C N C N C C C A T T T C T A C C G C N G A G T G C N N A C C C N G T A N G G A 

Let's summarize the bases so we can look at things like the total number of `N` (bases which could not be determine).

In [239]:
sdf = pd.DataFrame(sequence_list, columns=['BASE'])
summary_sdf = pd.DataFrame(sdf['BASE'].value_counts())
summary_sdf.rename(columns={'BASE': 'Total'}, inplace=True)
summary_sdf['Percent of Total'] = summary_sdf['Total'] / summary_sdf['Total'].sum()

summary_sdf_style = summary_sdf.style.format({ 'Percent of Total': "{:.2%}"})
display(summary_sdf_style)

,Total,Percent of Total
N,148,24.54%
C,136,22.55%
T,123,20.40%
A,112,18.57%
G,84,13.93%


### Let's take a look at the Phred (_quality_) scores for the bases in the sequence.

In [153]:
%matplotlib widget

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

plt.style.use('seaborn-notebook')
plt.rcParams.update({ 'xtick.labelsize': 11 })

fig = plt.figure(figsize=(9,2.2))

# position the main graph and slider boxes
main_axis = plt.axes([0.05, 0.45, 0.9, 0.50]) # left bottom width height
slider_axis = plt.axes([0.17, 0.15, 0.75, 0.06])

quality_scores = chromatogram.letter_annotations['phred_quality']
valstep = 50

def build_bar_plot(start, end):
    main_axis.clear()

    seq = sequence_list[start:end]
    ticks = range(len(seq))
    
    main_axis.set_ylim(0)
    main_axis.set_xlim(0)
    main_axis.set_yticks([0, 20, 40, 60, 80])
    main_axis.set_xticks(ticks)
    main_axis.set_xticklabels(seq)
    
    main_axis.bar(ticks, quality_scores[start:end], align='center')
    plt.show()
    
# make the slider
q_slider = Slider(ax=slider_axis,
                  label='Click to Scroll', 
                  valmin=valstep, 
                  valmax=len(quality_scores),
                  valinit=valstep,
                  valstep=valstep)

def update(val):
    start = val - valstep
    build_bar_plot(start, val)
    
q_slider.on_changed(update)
build_bar_plot(0, valstep)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Note how the beginning and the ends of the quality scores taper off. Let's look at some general summary data of the quality scores.

In [147]:
quality_scores_df = pd.DataFrame(quality_scores)
quality_scores_df.describe()

,0
count,603.000000
mean,12.479270
std,5.736531
min,2.000000
25%,10.000000
50%,12.000000
75%,13.000000
max,43.000000
